<h1><center>  对比卖方分析师报告 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过www.sumup.ai获取 </center></h1>


 


## 目标: 
-	开发一条通道来比较和对比不同银行的研究分析师报告 

**SumUp对比分析的工作原理是用语料中两种不同类别的文档，由用户根据元数据或内容定义**

##  数据:
-	卖方研究报告，如股票研究或宏观研究

    **Nucleus Datafeed可用于主要中央银行和SEC文件的所有内容**


## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模API
 - 	*api_instance.post_topic_api(payload)*


-	对比主题建模API
 - 	*api_instance.post_topic_contrast_api(payload)*
 
 
-	文件对比摘要API
 - 	*api_instance.post_document_contrast_summary_api(payload)*

## 方法:

### 1.	数据集准备
-	创建包含所有相关文档的Nucleus数据集

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# Create API instance
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
folder = 'Sellside_research'         
dataset = 'Sellside_research'# str | 文件的目标数据集.

# 以递归方式从文件夹构建文件。  
# 其中每个项目采用以下格式：
# {'filename': filename,   # 要上传的文件名。必填
#  'metadata': {           # 该文件的元数据。选填
#      'key1': val1,       # 密钥就可以取任意名称，名字符合以下字符
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'company': 'Apple',
                                  'research_analyst': 'MS', # 修改此处的逻辑以提取给定报告的银行名称
                                  'date': '2019-01-01'}}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

### 2.	文件对比摘要
-    在这个例子中，我们将一类文档定义为摩根士丹利的研究分析报告。 第二类文件将包括所有其他研究报告

-    我们构建了对比汇总，将MS报告与其他银行的报告分开


-	使用以下输入参数来控制摘要的大小并过滤过短或过长的句子。
    - `summary_length`
    - `context_amount` (每个关键摘要句子周围的句子数)
    - `short_sentence_length`
    - `long_sentence_length`
    

-	设置以下参数以调整或细化对比摘要的焦点和内容
    - `custom_stop_words` (自定义停用词列表)
    - `syntax_variables` (包括/排除语法变量)
    - `compression` (控制对比度总结的广度)
    - `remove_redundancies` (删除冗余)


-	接下来，我们将讨论如何构建自定义的停用词列表以优化文档对比摘要

In [ ]:
print('---------------- Get doc contrasted summaries ------------------------')
metadata_selection = {"research_analyst": "MS"} # dict | 元数据选择定义了两类文档，以便相互对比和总结

query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）
custom_stop_words = ["morgan stanley"] # 停用词列表。 （可选）
summary_length = 6 # int | 用户在对比摘要中看到的最大点句数。 （可选）（默认为6）
context_amount = 0 # int | 围绕它们的关键摘要句子的句子数。 （可选）（默认为0）
short_sentence_length = 0 # int | 从摘要中排除的短句子长度（可选）（默认为4）
long_sentence_length = 40 # int | 从摘要中排除的长句子长度（可选）（默认为40）
excluded_docs = '' # str | 应从分析中排除的文档ID列表。 例如，[“docid1”，“docid2”，...，“docidN”]（可选）
syntax_variables = True # bool | 是否考虑每类文档的语法，语法的作用可以帮助对比（可选）（默认为False）
compression = 0.002 # float | 控制对比摘要广度的参数。 包含在0和1之间，它越小，捕获的对比度越多，重量越轻。 （可选）（默认为0.000002）
remove_redundancies = False # bool | 如果为True，则此选项从分析中删除准重复项，并仅重新获取其副本。 准复制将具有相同的NLP表示，但不一定是完全相同的文本。 （可选）（默认为False）

payload = nucleus_api.DocumentContrastSummaryModel(dataset="Sellside_research", 
                                                    metadata_selection=metadata_selection,
                                                    custom_stop_words=custom_stop_words,
                                                    period_start='2018-01-01',
                                                    period_end='2019-01-01')
try:
    api_response = api_instance.post_document_contrast_summary_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    print('Summary for', [x for x in  metadata_selection.values()])
    for sent in api_response.result.class_1_content.sentences:
        print('    *', sent)
    print('======')
    for sent in api_response.result.class_2_content.sentences:
        print('    *', sent)   

Extend the analysis to a list of sell-side coverages

In [ ]:
bank_list = ["MS", "GS", "JPM", "Citi", "BofA", "Barcap", "CS"]
custom_stop_words = ["morgan stanley", "goldman sachs", "jp morgan", "citigroup", "bank of america", "barclays", "credit suisse", "disclaimer", "disclosures"] # List of stop words. (optional)

query = '' # str | Dataset-language-specific全文查询，使用mysql MATCH boolean query格式（可选）

contrasts = []
for i in range(len(bank_list)):
    metadata_selection = {"research_analyst": bank_list[i]} # dict | T元数据选择定义了两类文档，以便相互对比和总结

    payload = nucleus_api.DocumentContrastSummaryModel(dataset="Sellside_research", 
                                                        metadata_selection=metadata_selection,
                                                        custom_stop_words=custom_stop_words,
                                                        period_start='2019-01-01',
                                                        period_end='2019-06-01')
    try:
        api_response = api_instance.post_document_contrast_summary_api(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:   
        contrasts.append({"bank": bank_list[i], "contrast":api_response.result.class_1_content.sentences})

### 3.	微调


#### a. 为您的对比主题指定`metadata_selection`

-     对比两个不同实体的文件

    从您自己的数据上，例如 卖方研究：

In [ ]:
metadata_selection = {"research_analyst": ["MS", "JPM"]}

    从SumUp自带数据上，例如， 中央银行：

In [ ]:
metadata_selection = {"bank": ["federal_reserve", "ECB"]}

-     对比给定实体的不同文档

    从SumUp自带数据上，例如， 中央银行：

In [ ]:
metadata_selection = {"document_category": ["speech", "press release"]}

-     对比包含不同关键字的文档

    从您自己的数据上，或在SumUp数据Feed上：

In [ ]:
metadata_selection = {"content": "fundamentals"}

#### b.	从对比摘要中排除某些内容
-   通过使用Doc Contrasted Summary API中的`custom_stop_words`参数，排除无关的关键字/主题以定制对比度分析


-	在语料库中的文档中提取对比主题的关键字，并打印这些主题的关键字



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Sellside_research',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection)
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

使用您的专业知识，可以再亲自确定特定主题或关键字是否有足够的区别度来完成文档对比摘要。

然后，您可以通过创建包含这些单词的`custom_stop_words`变量来定制文档对比摘要。 如下所示，用这个初始化变量并将其传递到第2节主代码的有效负载（payload）中：

In [1]:
custom_stop_words = ["disclaimer","disclosure"] # str | 停用词列表。 （可选）

#### c. 将对比摘要集中在您的语料库中可能讨论的特定主题上
**query**: 将对比摘要集中在您的语料库中可能讨论的特定主题上

使用特定查询或查询重新运行Contrast Analysis APIs。 创建一个变量`query`并将其传递给有效负载(payload)：

In [ ]:
query = '(earnings OR cash flows)' # str | 全文查询，使用mysql MATCH boolean query格式。 示例：“（word1 OR word2）AND（word3 OR word4）”（可选）

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于Sumup Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于Sumut Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到Sumup Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 